# TP2 - Spark

In [1]:
%config Completer.use_jedi = False
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
from nltk.util import ngrams
import nltk
import pandas as pd

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sqlContext = SQLContext(sc)

/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Consulta 11
Generar una función genérica que dado un n nos permita obtener un RDD con los
n-gramas del contenido de texto de wikipedia y su frecuencia

In [3]:
def wikiNgramsFreq(n):
    # hago esto para poder leer el csv correctamente (propuesto por Gian)
#     texts = pd.read_csv('/mnt/882C57D72C57BEBE/gabriel/FIUBA/ORGADATOS/TP1/data/contents_text_sample.csv')
#     texts.to_parquet('contents_text_sample.parquet')
#     del texts
    dfText = sqlContext.read.parquet('contents_text_sample.parquet')
    rddText = dfText.rdd
    # obtengo los ngramas con sus frecuencias
    return rddText.flatMap(lambda entrada: ngrams(str(entrada.text).lower().split(), n) ).map(lambda ngram : (ngram, 1)).reduceByKey(lambda a,b : a+b)

In [4]:
# pruebo el funcionamiento de la función
freq4 = wikiNgramsFreq(4)
freq4.take(10)

[(('la', 'isla', 'de', '[[eubea]]'), 4),
 (('por', 'david', 'ritz.', "''the"), 1),
 (('a', 'cabo', 'la', 'celebración'), 2),
 (('necesarios', '||', '26.6.2006', '||'), 1),
 (('united', 'against', 'apartheid]],', 'que'), 1),
 (('villa,', 'destacando', 'las', 'de'), 1),
 (('la', 'associazione', 'gruppi', 'di'), 1),
 (('se', 'alinearon', 'con', 'el'), 1),
 (('las',
   'sub-23',
   '|url=http://masdeporte.as.com/masdeporte/2012/07/25/juegosolimpicos/1343173475_894631.html',
   '|fechaacceso=24'),
  1),
 (('ornstein', 'como', 'coautor', 'de'), 1)]

## Consulta 13
La región por cada país que tiene la mayor cantidad de contenidos publicados.

In [18]:
dfgeo = sqlContext.read.csv('/mnt/882C57D72C57BEBE/gabriel/FIUBA/ORGADATOS/TP1/data/geo_tags.csv', header=True, inferSchema=True)
rddgeo = dfgeo.rdd
rddgeo.take(2)

[Row(gt_id='158041', gt_page_id=4328020, gt_globe='earth', gt_primary=0, gt_lat=41.50749969, gt_lon=-5.81750011, gt_dim=1000, gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844377', gt_page_id=4670424, gt_globe='earth', gt_primary=0, gt_lat=13.73799992, gt_lon=-89.29199982, gt_dim=10000, gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

In [19]:
rddgeo = rddgeo.filter(lambda x : ((x.gt_country != None) and (x.gt_region != None)))
rddgeo.take(2)

[Row(gt_id='41228448', gt_page_id=1515657, gt_globe='earth', gt_primary=0, gt_lat=-31.475, gt_lon=-68.65, gt_dim=10000, gt_type='waterbody', gt_name=None, gt_country='AR', gt_region='J'),
 Row(gt_id='91697926', gt_page_id=8623414, gt_globe='earth', gt_primary=0, gt_lat=42.8475, gt_lon=-2.67222222, gt_dim=1000, gt_type='landmark', gt_name='Palacio Villa de Suso', gt_country='ES', gt_region='MC')]

In [20]:
# cuento la cantidad de apariciones de cada par pais-ciudad (por las dudas de que haya ciudades con igual abreviacion para paises distintos)
regiones = rddgeo.map(lambda x : ((x.gt_country, x.gt_region), 1) ).reduceByKey(lambda a,b : a+b)
regiones.take(10)

[(('GB', 'PTE'), 1),
 (('GB', 'STS'), 3),
 (('JM', '1'), 2),
 (('MX', 'QUE'), 12),
 (('RO', 'SB'), 1),
 (('TR', '34'), 6),
 (('DE', 'HE'), 19),
 (('US', 'IL'), 19),
 (('CA', 'BC'), 18),
 (('MX', 'YUC'), 6)]

In [21]:
# reordeno la clave-valor y me quedo solo con la ciudad de cada pais que tenga mas apariciones
topregiones = regiones.map(lambda x : (x[0][0], (x[0][1], x[1])))
topregiones = topregiones.reduceByKey(lambda a,b: a if (a[1]>=b[1]) else b)
#hasta acá es lo pedido por la consigna

In [22]:
# lo ordeno por curiosidad
topregiones.takeOrdered(30, key = lambda x : -x[1][1])

[('ES', ('CT', 300)),
 ('US', ('CA', 180)),
 ('CA', ('QC', 173)),
 ('JP', ('13', 104)),
 ('DE', ('BE', 51)),
 ('FI', ('ES', 27)),
 ('IT', ('RM', 21)),
 ('AT', ('9', 17)),
 ('AR', ('C', 14)),
 ('MX', ('QUE', 12)),
 ('CH', ('BS', 10)),
 ('CN', ('11', 10)),
 ('GB', ('WSM', 10)),
 ('AU', ('WA', 9)),
 ('FR', ('75', 8)),
 ('IE', ('L', 8)),
 ('NI', ('MN', 8)),
 ('PE', ('PAS', 7)),
 ('BR', ('ES', 7)),
 ('TR', ('34', 6)),
 ('SE', ('AB', 6)),
 ('RU', ('MOW', 5)),
 ('DK', ('82', 5)),
 ('BE', ('VOV', 5)),
 ('IN', ('DL', 4)),
 ('IL', ('JM', 4)),
 ('BA', ('BIH', 4)),
 ('AZ', ('ABS', 4)),
 ('PL', ('MA', 3)),
 ('KP', ('8', 3))]

## Consulta 14
El Top 5 de contenidos que tienen la mayor cantidad de redirecciones que apuntan a
ellos

In [3]:
dflinks = sqlContext.read.csv('/mnt/882C57D72C57BEBE/gabriel/FIUBA/ORGADATOS/TP1/data/redirect_list.csv', header=True, inferSchema=True)
rddlinks = dflinks.rdd
rddlinks.take(5)

[Row(rd_from=54, rd_namespace=0, rd_title='Apiales', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=57, rd_namespace=0, rd_title='Atmósfera', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=58, rd_namespace=0, rd_title='Archaea', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=82, rd_namespace=0, rd_title='Agujero_de_gusano', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=108, rd_namespace=0, rd_title='Stipa', rd_interwiki=None, rd_fragment=None)]

In [4]:
redcontents = rddlinks.map(lambda x : (x.rd_title, 1))
# obtengo el top5 de contenidos con mayor cantidad de redirecciones hacia ellos
redcontents.reduceByKey(lambda a,b : a+b).takeOrdered(5, lambda x: -x[1])

[('Artículo_futuro', 1161),
 ('IV_milenio', 1001),
 ('V_milenio', 998),
 ('Estaciones_de_Metrobús_de_la_Ciudad_de_México', 396),
 ('163.117.0.0', 310)]

## Consulta 35
¿Cuál es el mínimo que ha durado desde su registro un usuario bloqueado en la
plataforma?

In [3]:
# hago esto para poder leer el csv correctamente (propuesto por Gian)
# logs = pd.read_csv('/mnt/882C57D72C57BEBE/gabriel/FIUBA/ORGADATOS/TP1/data/logs.csv')
# logs.to_parquet('logs.parquet')
# del logs
dflogs = sqlContext.read.parquet('logs.parquet')
rddlogs = dflogs.rdd
rddlogs.take(5)

[Row(item_id=1, timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id=2, timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel'),
 Row(item_id=3, timestamp='2004-12-23T07:26:10Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '#REDIRECT [[Flash]]'", logtype='delete', action='delete', title='Macromedia Flash'),
 Row(item_id=4, timestamp='2004-12-23T07:55:09Z', contributor_username='Sanbec', contributor_id=7856.0, contributor_ip=None, comment='Sin información de origen ni de 

In [4]:
# me quedo con los nombres de usuarios que fueron bloqueados y la fecha de bloqueo
rddblocks = rddlogs.filter(lambda x : (x.action == 'block')).map(lambda linea : (str(linea.title).split(':',1), linea.timestamp)).filter(lambda x: len(x[0]) > 1)\
    .map(lambda x : (x[0][1], x[1]) ).cache()
rddblocks.takeSample(False, 10)

[('152.26.209.40', '2016-11-06T23:26:09Z'),
 ('80.34.175.241', '2012-05-15T13:15:09Z'),
 ('2806:108E:1A:1707:ECB7:F1EC:FECC:BFE7', '2019-06-18T09:07:28Z'),
 ('80.103.58.67', '2010-02-16T03:04:18Z'),
 ('181.196.227.142', '2014-11-04T21:36:48Z'),
 ('Millavy', '2011-08-31T17:16:40Z'),
 ('200.28.135.70', '2009-11-20T12:09:31Z'),
 ('190.51.104.26', '2007-12-17T14:37:59Z'),
 ('190.149.77.229', '2012-11-15T16:12:32Z'),
 ('200.54.105.49', '2018-09-04T17:48:58Z')]

In [5]:
# me quedo con los nombres de usuarios nuevos y su fecha de creacion
rddnewacc = rddlogs.filter(lambda x : (x.logtype == 'newusers' and x.action == 'create')).map(lambda x: (x.contributor_username, x.timestamp)).cache()
rddnewacc.takeSample(False, 10)

[('Thepartyofspain', '2013-05-24T22:49:16Z'),
 ('JOSÉ ESCOBAR', '2007-05-24T01:13:23Z'),
 ('Vjimenezvilla', '2020-04-18T15:08:46Z'),
 ('Dayanaquinrero', '2016-08-20T01:43:57Z'),
 ('JhosuaDJ', '2015-06-01T03:12:45Z'),
 ('Anjhusabu', '2006-10-07T00:24:42Z'),
 ('Ga06', '2014-10-15T17:48:37Z'),
 ('Lujan martinez', '2016-08-03T02:54:17Z'),
 ('Funcion.gamma', '2007-06-23T02:00:50Z'),
 ('La chica ffaso', '2013-05-26T23:38:38Z')]

In [6]:
# me quedo solo con el 1er registro y 1er bloqueo por cada usuario
rddnewacc = rddnewacc.reduceByKey(lambda date1, date2 : date1 if pd.to_datetime(date1, format = '%Y-%m-%dT%H:%M:%SZ') <= pd.to_datetime(date2, format = '%Y-%m-%dT%H:%M:%SZ') else date2)
rddblocks = rddblocks.reduceByKey(lambda date1, date2 : date1 if pd.to_datetime(date1, format = '%Y-%m-%dT%H:%M:%SZ') <= pd.to_datetime(date2, format = '%Y-%m-%dT%H:%M:%SZ') else date2)

In [7]:
rdduserduration = rddnewacc.join(rddblocks).cache()
rdduserduration.take(10)

[('Bloquearmeeeeeeeeeee', ('2006-05-26T14:53:24Z', '2019-08-25T22:55:06Z')),
 ('Cagabot', ('2006-06-09T03:58:12Z', '2006-06-09T04:04:14Z')),
 ("D'Alembert~eswiki", ('2006-12-31T03:09:25Z', '2007-01-04T15:56:18Z')),
 ('Dekadente', ('2007-01-29T22:27:39Z', '2007-01-31T20:57:04Z')),
 ('Javilondio7', ('2007-03-19T20:09:43Z', '2007-05-31T00:16:57Z')),
 ('CHOY SAN SANTA VICTORIA', ('2007-05-26T03:07:35Z', '2007-05-26T03:19:01Z')),
 ('Sonvalidoslosnombresconcomillas"""',
  ('2007-05-28T21:04:48Z', '2007-05-28T21:05:18Z')),
 ('Rafax1984', ('2007-06-18T14:48:59Z', '2007-09-01T00:35:08Z')),
 ('Aknaxedon', ('2007-07-04T15:38:11Z', '2009-07-06T13:32:32Z')),
 ('L678277198', ('2007-09-05T17:57:47Z', '2007-09-09T15:21:10Z'))]

In [8]:
# calculo la duracion de los usuarios hasta ser bloqueados
rdduserduration = rdduserduration.map(lambda x : (x[0], pd.to_datetime(x[1][1], format = '%Y-%m-%dT%H:%M:%SZ') - pd.to_datetime(x[1][0], format = '%Y-%m-%dT%H:%M:%SZ'))).cache()
rdduserduration.take(20)

[('Bloquearmeeeeeeeeeee', Timedelta('4839 days 08:01:42')),
 ('Cagabot', Timedelta('0 days 00:06:02')),
 ("D'Alembert~eswiki", Timedelta('4 days 12:46:53')),
 ('Dekadente', Timedelta('1 days 22:29:25')),
 ('Javilondio7', Timedelta('72 days 04:07:14')),
 ('CHOY SAN SANTA VICTORIA', Timedelta('0 days 00:11:26')),
 ('Sonvalidoslosnombresconcomillas"""', Timedelta('0 days 00:00:30')),
 ('Rafax1984', Timedelta('74 days 09:46:09')),
 ('Aknaxedon', Timedelta('732 days 21:54:21')),
 ('L678277198', Timedelta('3 days 21:23:23')),
 ('Negracandelaria', Timedelta('1494 days 14:49:47')),
 ('Fucktomas', Timedelta('0 days 04:08:22')),
 ('Miguel-rubio-10', Timedelta('0 days 00:06:40')),
 ('Miik Ezdanitofff', Timedelta('115 days 19:29:15')),
 ('El q me banea es gay', Timedelta('0 days 00:05:20')),
 ('Inexistente', Timedelta('0 days 00:08:51')),
 ('Congregacion Cristiana Cubana', Timedelta('5 days 01:28:57')),
 ('Changosferch123', Timedelta('0 days 00:49:13')),
 ('Demian22Chito', Timedelta('7 days 23:50:

In [9]:
# obtengo el usuario que menos tiempo duró hasta ser bloqueado
rdduserduration.filter(lambda x: x[1] > pd.to_timedelta(0)).reduce(lambda a,b : a if a[1] <= b[1] else b)

('Unfollow Ariiana Grande 03.07.2021 7', Timedelta('0 days 00:00:04'))

## Consulta 36
La 3-upla de palabras más común en los comentarios de los logs

In [4]:
# hago esto para poder leer el csv correctamente (propuesto por Gian)
# logs = pd.read_csv('/mnt/882C57D72C57BEBE/gabriel/FIUBA/ORGADATOS/TP1/data/logs.csv')
# logs.to_parquet('logs.parquet')
# del logs
dflogs = sqlContext.read.parquet('logs.parquet')
rddlogs = dflogs.rdd

# uso lo que hice en el punto 11
trigram = rddlogs.flatMap(lambda entrada: ngrams(str(entrada.comment).lower().split(), 3) ).map(lambda ngram : (ngram, 1)).reduceByKey(lambda freq1,freq2 : freq1+freq2).reduce(lambda a,b : a if a[1] >= b[1] else b )
trigram

/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(('página', 'creada', 'con'), 681872)

## Consulta 21
El contenido con mayor cantidad de de acciones realizadas para todos los tipos
posibles de acciones

In [3]:
# logs = pd.read_csv('/mnt/882C57D72C57BEBE/gabriel/FIUBA/ORGADATOS/TP1/data/logs.csv')
# logs.to_parquet('logs.parquet')
# del logs
dflogs = sqlContext.read.parquet('logs.parquet')
rddlogs = dflogs.rdd
rddlogs.take(3)

/mnt/882C57D72C57BEBE/Anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


[Row(item_id=1, timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id=2, timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel'),
 Row(item_id=3, timestamp='2004-12-23T07:26:10Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '#REDIRECT [[Flash]]'", logtype='delete', action='delete', title='Macromedia Flash')]

In [5]:
dfcontents = sqlContext.read.csv('/mnt/882C57D72C57BEBE/gabriel/FIUBA/ORGADATOS/TP1/data/contents.csv', header=True, inferSchema=True)
rddcontents = dfcontents.rdd
rddcontents.take(3)

[Row(title='Wikipedia:Artículos solicitados', id='5', namespace='4', revision_id='132533307', parent_revision='132516631', revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id='1708233', revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id='7', namespace='0', revision_id='138035057', parent_revision='138027475', revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id='10', namespace='0', revision_id='137966826', parent_revision='137825165', revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id='4642283', revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»')]

In [4]:
# obtengo las tuplas (titulo del log, accion)
items = rddlogs.map(lambda x: (x.title, x.action)).cache()
items.take(10)

[('Verificacionismo', 'delete'),
 ('Rapunzel', 'delete'),
 ('Macromedia Flash', 'delete'),
 ('Archivo:Gnu head.png', 'delete'),
 ('Archivo:Gnu-head-sm.jpg', 'delete'),
 ('Usuario:212.55.30.157', 'block'),
 ('Archivo:La florida.jpg', 'upload'),
 ('Archivo:La florida uoct.jpg', 'upload'),
 ('Archivo:Pablo zalaquett.jpg', 'upload'),
 ('Archivo:Szymborska znaczek.jpg', 'delete')]

In [6]:
# obtuve las tuplas (titulo del contenido, 1) ahora porque si queria agregar el 1 después para contar apariciones tenia errores y esta me pareció una buena forma de solucionarlo
titulos = rddcontents.map(lambda x: (x.title, 1)).cache()
titulos.take(10)

[('Wikipedia:Artículos solicitados', 1),
 ('Andorra', 1),
 ('Argentina', 1),
 ('Geografía de Andorra', 1),
 ('Demografía de Andorra', 1),
 ('Comunicaciones de Andorra', 1),
 ('Artes visuales', 1),
 ('Agricultura', 1),
 ('Astronomía galáctica', 1),
 ('ASCII', 1)]

In [7]:
# me quedo solo con los titulos que corresponden a contenidos y su accion
contenidos = items.join(titulos).cache()
contenidos.take(10)

[('Geografía de Kiribati', ('delete', 1)),
 ('Joonia', ('delete', 1)),
 ('Joonia', ('delete', 1)),
 ('Voto electrónico', ('delete', 1)),
 ('Isidro Nonell', ('delete', 1)),
 ('Isidro Nonell', ('delete', 1)),
 ('Isidro Nonell', ('move_redir', 1)),
 ('Isidro Nonell', ('delete_redir', 1)),
 ('Primera Epístola a Timoteo', ('delete', 1)),
 ('Primera Epístola a Timoteo', ('delete', 1))]

In [8]:
# cuento la cantidad de veces que se produjo cada par (titulo, accion), osea cantidad de veces que se le aplicó esa accion a ese contenido
contenidos = contenidos.map(lambda x : ((x[0],x[1][0]), x[1][1])).reduceByKey(lambda a,b : a+b).cache()
contenidos.take(10)

[(('Primera Epístola a Timoteo', 'move'), 1),
 (('Tsubaki', 'move'), 1),
 (('Nombre chino', 'move'), 1),
 (('Club Atlético San Lorenzo de Almagro', 'unprotect'), 1),
 (('Luco', 'move'), 1),
 (('Tigre (Buenos Aires)', 'move'), 1),
 (('Nacho', 'move'), 1),
 (('San Próspero', 'move'), 1),
 (('Osorno', 'move'), 5),
 (('Equus asinus', 'move'), 2)]

In [9]:
# elimino los que no tienen titulo y reordeno las entradas de forma (accion, (titulo, cantidad))
contenidos = contenidos.filter(lambda x: x[0][0] != None).map(lambda x : (x[0][1], (x[0][0], x[1]))).cache()
contenidos.take(10)

[('move', ('Primera Epístola a Timoteo', 1)),
 ('move', ('Tsubaki', 1)),
 ('move', ('Nombre chino', 1)),
 ('unprotect', ('Club Atlético San Lorenzo de Almagro', 1)),
 ('move', ('Luco', 1)),
 ('move', ('Tigre (Buenos Aires)', 1)),
 ('move', ('Nacho', 1)),
 ('move', ('San Próspero', 1)),
 ('move', ('Osorno', 5)),
 ('move', ('Equus asinus', 2))]

In [10]:
# me quedo solo con los titulos con masyor cantidad por cada accion y ordeno las entradas de mayor a menor cantidad de apariciones para que se vea mas lindo :) 
contenidos.reduceByKey(lambda a,b : a if a[1] >= b[1] else b).takeOrdered(10, lambda x: -x[1][1])

[('update', ('Wikipedia:Portada', 513)),
 ('revision', ('Wikipedia:Vandalismo en curso', 310)),
 ('delete', ('Anexo:Wikipedias', 100)),
 ('restore', ('Plantilla:Ficha de entidad subnacional', 64)),
 ('modify',
  ('Wikipedia:Tablón de anuncios de los bibliotecarios/Portal/Archivo/Miscelánea/Actual',
   61)),
 ('move', ('Plantilla:Ficha de entidad subnacional', 59)),
 ('move_prot', ('Plantilla:Ficha de entidad subnacional', 50)),
 ('protect',
  ('Wikipedia:Tablón de anuncios de los bibliotecarios/Portal/Archivo/Solicitudes de nueva consulta/Actual',
   46)),
 ('add', ('Testimonio', 31)),
 ('create', ('Himawari Uzumaki', 15))]